# Random Baseline Cartpole 

In [1]:
import gymnasium as gym
from pathlib import Path
from gymnasium.wrappers import RecordVideo
from datetime import datetime
from stable_baselines3.common.monitor import Monitor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
docs_path = Path("../../documentation/cartpole/random-baseline") # ../ makes it so it writes to a directory one back from current one

run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_path = docs_path / f"run_{run_id}"

video_dir = run_path / "videos"
graphs_dir  = run_path / "graphs"
report_file = run_path / "random_baseline.md"
monitor_dir = run_path / "monitor"

docs_path.mkdir(parents=True, exist_ok=True)
graphs_dir.mkdir(parents=True, exist_ok=True)
docs_path.mkdir(parents=True, exist_ok=True)
video_dir.mkdir(parents=True, exist_ok=True)
monitor_dir.mkdir(parents=True, exist_ok=True)

monitor_file = str(monitor_dir) #/ "monitor_log.csv"


In [12]:
def run_random_baseline_with_monitor(video_dir, monitor_file, num_episodes=100, seed=42):
    print("\nRunning Random Baseline logging rewards per episode...")

    monitor_path = Path(monitor_file)
    if monitor_path.is_dir():
        monitor_path = monitor_path

    env = gym.make("CartPole-v1", render_mode="rgb_array")
    env = Monitor(env, str(monitor_path))
    env = RecordVideo(
        env,
        video_folder=str(video_dir),
        episode_trigger=lambda e: e < 5,
        name_prefix="cartpole_random_baseline"
    )
    labels = ["cart position", "cart velocity", "pole angle", "pole angular velocity"]
    # Reset environment
    observation, info = env.reset(seed=42)
    
    rewards = []
    for ep in range(num_episodes):
        obs, info = env.reset(seed=seed + ep)
        done = False
        total_reward = 0
        step = 0
        print(f"Episode {ep}: (seed = {seed+ep})")
        while not done:
            step += 1
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            total_reward += reward

            print(f"Step {step}:")
            print(f"Action taken: {action}")
            for label, observe in zip(labels,observation):
                print(f"{label}: {observe}")
            print(f"Reward: {reward}")
            print(f"Terminated: {terminated}, Truncated: {truncated}")
            print("-" * 50)
            
        rewards.append(total_reward)
        print(f"Episode {ep}: reward = {total_reward}")

        if ep == 67:
            log_episode = {
                "episode": ep,
                "seed": seed + ep,
                "action": action,
                "observation":observation,
                "reward":total_reward
            }

    env.close()
    print(f"Logged rewards to {monitor_path}")
    return np.mean(rewards), log_episode
    


In [13]:
def plot_random_baseline_curve(monitor_file, graphs_dir):
    monitor_path = Path(monitor_file)

    if monitor_path.is_dir():
        monitor_path = monitor_path/ "monitor.csv"

    if not monitor_path.exists():
        raise FileNotFoundError(f"No monitor file found at {monitor_path}")

    df = pd.read_csv(monitor_path, skiprows=1)
    plt.figure(figsize=(10, 6))
    plt.plot(df["r"], label="Reward per Episode", color="tab:red", alpha=0.7)
    plt.xlabel("Episodes")
    plt.ylabel("Total Reward")
    plt.title("Random Baseline Learning Curve (CartPole-v1)")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()
    out = Path(graphs_dir) / "random_baseline_curve.png"
    plt.savefig(out, dpi=200, bbox_inches="tight")
    plt.close()

    print(f"Random baseline curve saved → {out}")



In [14]:
env = gym.make("CartPole-v1", render_mode="rgb_array") 
env = RecordVideo(
    env,
    video_folder=str(video_dir),
    episode_trigger=lambda e: True,
    name_prefix="cartpole_random_baseline" 
)

/Users/davidjayakumar/Library/CloudStorage/OneDrive-AtlanticTU/YEAR 4/Project Engineering/Network-Defender/.venv_mac/lib/python3.9/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /Users/davidjayakumar/Library/CloudStorage/OneDrive-AtlanticTU/YEAR 4/Project Engineering/Network-Defender/documentation/cartpole/random-baseline/run_2025-10-12_13-50-18/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [15]:

total_reward = 0
episode_over = False

In [16]:
average_reward,log_episode = run_random_baseline_with_monitor(video_dir, monitor_file, num_episodes=100, seed=42)


Running Random Baseline logging rewards per episode...
Episode 0: (seed = 42)
Step 1:
Action taken: 1
cart position: 0.02739560417830944
cart velocity: -0.006112155970185995
pole angle: 0.03585979342460632
pole angular velocity: 0.019736802205443382
Reward: 1.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 2:
Action taken: 1
cart position: 0.02739560417830944
cart velocity: -0.006112155970185995
pole angle: 0.03585979342460632
pole angular velocity: 0.019736802205443382
Reward: 1.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 3:
Action taken: 0
cart position: 0.02739560417830944
cart velocity: -0.006112155970185995
pole angle: 0.03585979342460632
pole angular velocity: 0.019736802205443382
Reward: 1.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 4:
Action taken: 1
cart position: 0.02739560417830944
cart velocity: -0.006112155970185995
pole angle: 

In [17]:
obs_explanation = """\
**Observation vector (4 values):**
1. **Cart Position (m)** — horizontal position on the track (≈ -4.8 to +4.8).
2. **Cart Velocity (m/s)** — how fast the cart moves (unbounded float in practice).
3. **Pole Angle (rad)** — tilt of the pole relative to vertical (≈ -0.4189 to +0.4189 rad ≈ ±24°).
4. **Pole Angular Velocity (rad/s)** — how fast the pole is rotating (unbounded float in practice).
"""

failure_conditions = """\
**Episode ends when (termination/truncation):**
- **Pole tilt exceeds ±0.4189 rad (~±24°)** → `terminated = True`
- **Cart position leaves track bounds (≈ ±4.8 m)** → `terminated = True`
- **Time limit of 500 steps is reached** → `truncated = True`
"""

with open(report_file, "w")as f:
    f.write("## Environment Details\n")
    f.write(f"- Action space: {env.action_space}\n")
    f.write(f"- Observation space: {env.observation_space}\n")
    f.write(f"- Maximum steps per episode: {env.spec.max_episode_steps}\n\n")

    f.write("## Observation Meaning\n")
    f.write(obs_explanation + "\n")

    f.write("## Failure Conditions\n")
    f.write(failure_conditions + "\n")

    f.write("## Example Episode (From the 100 Runs)\n")
    f.write(f"- Episode: {log_episode['episode']}\n")
    f.write(f"- Seed: {log_episode['seed']}\n")
    f.write(f"- Observation: {log_episode['observation']}\n")
    f.write(f"- Total Reward: {log_episode['reward']}\n\n")

    f.write("## 100 episode Performance Summary:\n")
    f.write(f"- Average reward: {average_reward}\n\n")

In [18]:
plot_random_baseline_curve(monitor_file, graphs_dir)

Random baseline curve saved → ../../documentation/cartpole/random-baseline/run_2025-10-12_13-50-18/graphs/random_baseline_curve.png
